# **Dự án phân tích dữ liệu thương mại điện tử - Làm sạch dữ liệu**

### Mục tiêu
- Xử lý các vấn đề trong bộ dữ liệu Brazilian E-Commerce Public Dataset (~100,000 dòng).
- Chuẩn bị dữ liệu cho phân tích (doanh thu, sản phẩm, hành vi khách hàng).

### Dữ liệu đầu vào
- `olist_orders_dataset.csv`: Thông tin đơn hàng (~99,441 dòng).
- `olist_order_items_dataset.csv`: Chi tiết sản phẩm (~112,650 dòng).

In [49]:
# Kiểm tra thông tin ban đầu
# Import thư viện cần thiết
import pandas as pd

# Đọc file CSV
orders = pd.read_csv('data/original_data/olist_orders_dataset.csv')
order_items = pd.read_csv('data/original_data/olist_order_items_dataset.csv')
products = pd.read_csv('data/original_data/olist_products_dataset.csv')

# Xem thông tin cơ bản
print("Thông tin orders:")
print(orders.info())
print("\nThông tin order_items:")
print(order_items.info())
print("\nThông tin products:")
print(products.info())

Thông tin orders:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB
None

Thông tin order_items:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0 

## **Bước 1: Kiểm tra lại dữ liệu**

In [50]:
# Kiểm tra kích thước và kiểu dữ liệu
# Kiểm tra kích thước
print("Kích thước orders:", orders.shape)
print("Kích thước order_items:", order_items.shape)
print("Kích thước products:", products.shape)

# Kiểm tra kiểu dữ liệu
print("\nKiểu dữ liệu orders:")
print(orders.dtypes)
print("\nKiểu dữ liệu order_items:")
print(order_items.dtypes)
print("\nKiểu dữ liệu products:")
print(products.dtypes)

Kích thước orders: (99441, 8)
Kích thước order_items: (112650, 7)
Kích thước products: (32951, 9)

Kiểu dữ liệu orders:
order_id                         object
customer_id                      object
order_status                     object
order_purchase_timestamp         object
order_approved_at                object
order_delivered_carrier_date     object
order_delivered_customer_date    object
order_estimated_delivery_date    object
dtype: object

Kiểu dữ liệu order_items:
order_id                object
order_item_id            int64
product_id              object
seller_id               object
shipping_limit_date     object
price                  float64
freight_value          float64
dtype: object

Kiểu dữ liệu products:
product_id                     object
product_category_name          object
product_name_lenght           float64
product_description_lenght    float64
product_photos_qty            float64
product_weight_g              float64
product_length_cm             float6

In [51]:
# Kiểm tra số lượng giá trị thiếu
print("Giá trị thiếu trong orders:")
print(orders.isnull().sum())
print("\nGiá trị thiếu trong order_items:")
print(order_items.isnull().sum())
print("\nGiá trị thiếu trong products:")
print(products.isnull().sum())

Giá trị thiếu trong orders:
order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

Giá trị thiếu trong order_items:
order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64

Giá trị thiếu trong products:
product_id                      0
product_category_name         610
product_name_lenght           610
product_description_lenght    610
product_photos_qty            610
product_weight_g                2
product_length_cm               2
product_height_cm               2
product_width_cm                2
dtype: int64


### Kết quả kiểm tra ban đầu
- **Số dòng/cột**: 
  - `orders`: (99441,8)
  - `order_items`: (112650, 7)
  - `products`: (32951, 9)
- **Kiểu dữ liệu**: 
  - `order_purchase_timestamp` hiện là object, cần chuyển thành datetime.
  - `price` và `freight_value` là float64, kiểm tra giá trị bất thường.
- **Vấn đề tiềm ẩn**: 
  - Giá trị thiếu trong bảng `orders`, và cột `product_category_name` của bảng `products`.
  - Có thể có giá trị âm hoặc quá cao trong `price`/`freight_value`.

## **Bước 2: Chuyển đổi kiểu dữ liệu và Xử lý giá trị thiếu**

### Mục tiêu
- Xử lý các giá trị thiếu trong bảng `orders`, và bảng `product`.
- Đảm bảo dữ liệu sẵn sàng cho phân tích.

### Ghi chú
- Theo kiểm tra ban đầu, `order_approved_at` có 160, `order_approved_at` có 1783, và `order_delivered_customer_date` có 2965 giá trị thiếu.
- Nguyễn nhân: Chủ yếu do tại thời điểm cập nhật, đơn hàng chưa đến bước xử lý đó
- Chiến lược: Điền giá trị dự kiến (ngày mua + 7 ngày) hoặc để trống tùy chọn.
- `product_category_name` có 610 giá trị thiếu -> Chiến lược dùng 'unknown' cho giá trị thiếu

In [52]:
#Chuyển đổi kiểu dữ liệu các cột thời gian
date_columns = [
    'order_purchase_timestamp',
    'order_approved_at',
    'order_delivered_carrier_date',
    'order_delivered_customer_date',
    'order_estimated_delivery_date'
]

# Chuyển kiểu dữ liệu các cột ngày sang datetime
for col in date_columns:
    orders[col] = pd.to_datetime(orders[col])

order_items['shipping_limit_date'] = pd.to_datetime(order_items['shipping_limit_date'], errors='coerce')
orders.info()  # để xác nhận kiểu dữ liệu đã đổi
order_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  object        
 1   customer_id                    99441 non-null  object        
 2   order_status                   99441 non-null  object        
 3   order_purchase_timestamp       99441 non-null  datetime64[ns]
 4   order_approved_at              99281 non-null  datetime64[ns]
 5   order_delivered_carrier_date   97658 non-null  datetime64[ns]
 6   order_delivered_customer_date  96476 non-null  datetime64[ns]
 7   order_estimated_delivery_date  99441 non-null  datetime64[ns]
dtypes: datetime64[ns](5), object(3)
memory usage: 6.1+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype     

In [60]:
# Điền giá trị thiếu
# 1. Tính khoảng cách trung bình (mean timedelta) từ các bản ghi đủ dữ liệu
delta_approved = (orders['order_approved_at'] - orders['order_purchase_timestamp']).mean()
delta_carrier = (orders['order_delivered_carrier_date'] - orders['order_approved_at']).mean()
delta_customer = (orders['order_delivered_customer_date'] - orders['order_delivered_carrier_date']).mean()
delta_estimated = (orders['order_estimated_delivery_date'] - orders['order_approved_at']).mean()

# 2. Điền giá trị thiếu 
orders.loc[orders['order_approved_at'].isna(), 'order_approved_at'] = orders['order_purchase_timestamp'] + delta_approved

orders.loc[orders['order_delivered_carrier_date'].isna(), 'order_delivered_carrier_date'] = orders['order_approved_at'] + delta_carrier

orders.loc[orders['order_delivered_customer_date'].isna(), 'order_delivered_customer_date'] = orders['order_delivered_carrier_date'] + delta_customer

orders.loc[orders['order_estimated_delivery_date'].isna(), 'order_estimated_delivery_date'] = orders['order_approved_at'] + delta_estimated

products['product_category_name'] = products['product_category_name'].fillna('unknown')

orders.info()  # để xác nhận kiểu dữ liệu đã đủ
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  object        
 1   customer_id                    99441 non-null  object        
 2   order_status                   99441 non-null  object        
 3   order_purchase_timestamp       99441 non-null  datetime64[ns]
 4   order_approved_at              99441 non-null  datetime64[ns]
 5   order_delivered_carrier_date   99441 non-null  datetime64[ns]
 6   order_delivered_customer_date  99441 non-null  datetime64[ns]
 7   order_estimated_delivery_date  99441 non-null  datetime64[ns]
dtypes: datetime64[ns](5), object(3)
memory usage: 6.1+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype 

## **Bước 3: Kiểm tra và xử lý bất thường**

### Mục tiêu
- Phát hiện các giá trị bất thường (outliers) trong `price` và `freight_value` của `order_items`.
- Loại bỏ hoặc điều chỉnh các giá trị không hợp lý (ví dụ: giá âm, quá cao).

### Ghi chú
- Giá trị bất thường có thể là giá âm, giá quá cao (ví dụ: >10,000), hoặc phí vận chuyển bất thường.
- Sử dụng ngưỡng hợp lý dựa trên phân phối dữ liệu.

In [54]:
# Thống kê mô tả cho price và freight_value
print("Thống kê price:")
print(order_items['price'].describe())
print("\nThống kê freight_value:")
print(order_items['freight_value'].describe())

Thống kê price:
count    112650.000000
mean        120.653739
std         183.633928
min           0.850000
25%          39.900000
50%          74.990000
75%         134.900000
max        6735.000000
Name: price, dtype: float64

Thống kê freight_value:
count    112650.000000
mean         19.990320
std          15.806405
min           0.000000
25%          13.080000
50%          16.260000
75%          21.150000
max         409.680000
Name: freight_value, dtype: float64


### Kết quả kiểm tra ban đầu
- `price`: Giá tối đa 6735.00, giá tối thiểu 0.85.
- `freight_value`: Phí vận chuyển tối đa 409.68, phí tối thiểu 0.00.
- **Quan sát**: Không có giá âm hoặc quá lớn (do lỗi nhập liệu), giá bán > 0, phí vận chuyển có thể bằng 0 (free shipping) -> không cần loại bỏ outline.

## **Bước 4: Kiểm tra và xử lý trùng lặp**

### Mục tiêu
- Kiểm tra xem có dòng trùng lặp trong `orders` và `order_items` không.
- Loại bỏ các dòng trùng lặp để đảm bảo dữ liệu không bị nhân đôi.

### Ghi chú
- Trùng lặp có thể xảy ra do lỗi nhập liệu hoặc sao chép dữ liệu.
- Sử dụng phương pháp giữ dòng đầu tiên khi xóa trùng lặp.

In [61]:
# Kiểm tra số lượng dòng trùng lặp
duplicates_orders = orders.duplicated().sum()
duplicates_order_items = order_items.duplicated().sum()
duplicates_products = products.duplicated().sum()

print("Số dòng trùng lặp trong orders:", duplicates_orders)
print("Số dòng trùng lặp trong order_items:", duplicates_order_items)
print("Số dòng trùng lặp trong products:", duplicates_products)

Số dòng trùng lặp trong orders: 0
Số dòng trùng lặp trong order_items: 0
Số dòng trùng lặp trong products: 0


In [63]:
# Xóa dòng trùng lặp, giữ dòng đầu tiên
orders_cleaned = orders.drop_duplicates()
order_items_cleaned = order_items.drop_duplicates()
products_cleaned = products.drop_duplicates()

# Kiểm tra số dòng trước và sau khi xóa
print("Số dòng trong orders trước khi xóa:", orders.shape[0])
print("Số dòng trong orders sau khi xóa:", orders_cleaned.shape[0])
print("Số dòng trong order_items trước khi xóa:", order_items.shape[0])
print("Số dòng trong order_items sau khi xóa:", order_items_cleaned.shape[0])
print("Số dòng trong products trước khi xóa:", products.shape[0])
print("Số dòng trong products sau khi xóa:", products_cleaned.shape[0])

Số dòng trong orders trước khi xóa: 99441
Số dòng trong orders sau khi xóa: 99441
Số dòng trong order_items trước khi xóa: 112650
Số dòng trong order_items sau khi xóa: 112650
Số dòng trong products trước khi xóa: 32951
Số dòng trong products sau khi xóa: 32951


In [65]:
# Cập nhật lại dữ liệu
orders = orders_cleaned
order_items = order_items_cleaned
products = products_cleaned

# Xác nhận không còn trùng lặp
print("Số dòng trùng lặp còn lại trong orders:", orders.duplicated().sum())
print("Số dòng trùng lặp còn lại trong order_items:", order_items.duplicated().sum())
print("Số dòng trùng lặp còn lại trong products:", products.duplicated().sum())

Số dòng trùng lặp còn lại trong orders: 0
Số dòng trùng lặp còn lại trong order_items: 0
Số dòng trùng lặp còn lại trong products: 0


## **Bước 5: Lưu dữ liệu đã làm sạch**


### Mục tiêu
- Lưu các bảng `orders` và `order_items` đã được làm sạch thành file CSV mới.
- Đảm bảo dữ liệu được bảo quản và sẵn sàng cho bước phân tích.

### Ghi chú
- File CSV mới sẽ được lưu trong thư mục `Ecommerce_Analysis\data\cleaned_data` với tên `cleaned_orders.csv` và `cleaned_order_items.csv`.
- Không lưu chỉ số hàng (`index=False`) để tránh cột thừa.

In [67]:
# Lưu dữ liệu đã làm sạch
orders.to_csv('data/cleaned_data/cleaned_orders.csv', index=False)
order_items.to_csv('data/cleaned_data/cleaned_order_items.csv', index=False)
products.to_csv('data/cleaned_data/cleaned_products.csv', index=False)

# Xác nhận file đã lưu
print("Đã lưu cleaned_orders.csv, cleaned_order_items.csv và cleaned_products.csv thành công.")

Đã lưu cleaned_orders.csv, cleaned_order_items.csv và cleaned_products.csv thành công.


In [68]:
# Đọc lại dữ liệu để kiểm tra
cleaned_orders = pd.read_csv('D:/Ecommerce_Analysis/data/cleaned_data/cleaned_orders.csv')
cleaned_order_items = pd.read_csv('D:/Ecommerce_Analysis/data/cleaned_data/cleaned_order_items.csv')
cleaned_products = pd.read_csv('D:/Ecommerce_Analysis/data/cleaned_data/cleaned_products.csv')

# Xem 5 dòng đầu
print("5 dòng đầu của cleaned_orders:")
display(cleaned_orders.head())
print("\n5 dòng đầu của cleaned_order_items:")
display(cleaned_order_items.head())
print("\n5 dòng đầu của cleaned_products:")
display(cleaned_products.head())

5 dòng đầu của cleaned_orders:


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15.000000000,2017-10-04 19:55:00.000000000,2017-10-10 21:25:13.000000000,2017-10-18
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27.000000000,2018-07-26 14:31:00.000000000,2018-08-07 15:27:45.000000000,2018-08-13
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23.000000000,2018-08-08 13:50:00.000000000,2018-08-17 18:06:29.000000000,2018-09-04
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59.000000000,2017-11-22 13:39:59.000000000,2017-12-02 00:28:42.000000000,2017-12-15
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29.000000000,2018-02-14 19:46:34.000000000,2018-02-16 18:17:02.000000000,2018-02-26



5 dòng đầu của cleaned_order_items:


,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14



5 dòng đầu của cleaned_products:


,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0
